## 📋 COMPREHENSIVE STOCK ANALYSIS SUMMARY

In [ ]:
import yfinance as yf
import pandas as pd
import os
import glob
from datetime import datetime
import numpy as np

In [ ]:
path = os.getcwd()

In [ ]:
# Get all CSV files with 'stock_data' in filename
stock_files = glob.glob("*stock_data*.csv")

print("Stock data files:")
for file in stock_files:
    print(f"  {file}")

In [ ]:
file_name = "stock_data_current_20251031_nuclear.csv"
file_path = path + "/" + file_name

In [ ]:
df = pd.read_csv(file_path, header = 0)
df = df.rename(columns={"Unnamed: 0": "Metrics"})
df = df.set_index('Metrics')

In [ ]:
# Calculate Price to Target Ratio (Current Price / Target Price) with proper float casting
df.loc['Price to Target Ratio'] = pd.to_numeric(df.loc['Current Price'], errors='coerce') / pd.to_numeric(df.loc['Target Price'], errors='coerce')

In [ ]:
# Let's first understand the complete dataset
print("📊 Dataset Overview:")
print(f"Total stocks: {len(df.columns)}")
print(f"Total metrics: {len(df.index)}")

In [ ]:
def filter_stocks_by_criteria(df, criteria):
    """
    Filter stocks based on multiple criteria
    
    Parameters:
    df: DataFrame from get_stock_metrics()
    criteria: dict of filtering criteria, e.g.:
        {
            'P/E Ratio TTM': {'max': 15, 'min': 1},
            'Current Ratio MRQ': {'min': 1.5},
            'ROE TTM': {'min': 0.15},
            'Debt to Equity MRQ': {'max': 50}
        }
    
    Returns:
    dict: {
        'filtered_tickers': list of tickers that meet all criteria,
        'results_summary': summary stats,
        'detailed_results': DataFrame with only filtered tickers
    }
    """
    filtered_tickers = []
    filter_results = {}
    
    for ticker in df.columns:
        meets_all_criteria = True
        ticker_results = {'ticker': ticker}
        
        # Check each criterion
        for metric, conditions in criteria.items():
            if metric not in df.index:
                ticker_results[f"{metric}_status"] = "metric_not_found"
                meets_all_criteria = False
                continue
                
            value = df.loc[metric, ticker]
            
            # Skip if no data (including NaN values)
            if pd.isna(value) or value == 'N/A' or value == 'NaN':
                ticker_results[f"{metric}_status"] = "no_data"
                meets_all_criteria = False
                continue
            
            # Convert to float if it's a string number
            try:
                if isinstance(value, str):
                    value = float(value)
            except (ValueError, TypeError):
                ticker_results[f"{metric}_status"] = "conversion_error"
                meets_all_criteria = False
                continue
            
            # Final check for numeric validity
            if not isinstance(value, (int, float)) or np.isnan(value):
                ticker_results[f"{metric}_status"] = "invalid_data"
                meets_all_criteria = False
                continue
                
            ticker_results[f"{metric}_value"] = value
            
            # Check min condition
            if 'min' in conditions and value < conditions['min']:
                ticker_results[f"{metric}_status"] = f"below_min_{conditions['min']}"
                meets_all_criteria = False
                continue
                
            # Check max condition  
            if 'max' in conditions and value > conditions['max']:
                ticker_results[f"{metric}_status"] = f"above_max_{conditions['max']}"
                meets_all_criteria = False
                continue
                
            ticker_results[f"{metric}_status"] = "pass"
        
        filter_results[ticker] = ticker_results
        
        if meets_all_criteria:
            filtered_tickers.append(ticker)
    
    # Create filtered DataFrame
    filtered_df = df[filtered_tickers] if filtered_tickers else pd.DataFrame()
    
    # Summary stats
    summary = {
        'total_tickers_analyzed': len(df.columns),
        'tickers_meeting_criteria': len(filtered_tickers),
        'success_rate': (len(filtered_tickers) / len(df.columns)) * 100 if df.columns.size > 0 else 0,
        'criteria_applied': criteria
    }
    
    return {
        'filtered_tickers': filtered_tickers,
        'results_summary': summary,
        'detailed_results': filtered_df,
        'filter_breakdown': filter_results
    }

In [ ]:
pd.set_option('display.max_rows', None)
df

In [ ]:
def comprehensive_stock_analysis(df):
    """
    Comprehensive stock analysis based on multiple financial health criteria
    
    Analysis Framework:
    
    [TO BE ADDED]
    
    Returns comprehensive analysis with tiered results
    """
    
    # Define criteria for different quality tiers
    excellent_criteria = {
      'P/E Ratio TTM': {'max': 22, 'min': 8},
      'P/B Ratio': {'max': 3.5, 'min': 0.5},
      'P/S Ratio TTM': {'max': 4, 'min': 0.2},
      'ROE TTM': {'min': 0.18},
      'Operating Margin MRQ': {'min': 0.15},
      'Profit Margin': {'min': 0.10},
      'Current Ratio MRQ': {'min': 1.8},
      'Debt to Equity MRQ': {'max': 50},
      'Revenue Growth YOY': {'min': 0.05}
    }
    
    good_criteria = {
      'P/E Ratio TTM': {'max': 25, 'min': 6},
      'P/B Ratio': {'max': 4, 'min': 0.3},
      'P/S Ratio TTM': {'max': 5, 'min': 0.1},
      'ROE TTM': {'min': 0.15},
      'Operating Margin MRQ': {'min': 0.12},
      'Profit Margin': {'min': 0.08},
      'Current Ratio MRQ': {'min': 1.5},
      'Debt to Equity MRQ': {'max': 60},
      'Revenue Growth YOY': {'min': 0.03}
    }
    
    acceptable_criteria = {
      'P/E Ratio TTM': {'max': 35, 'min': 4},
      'P/B Ratio': {'max': 6, 'min': 0.2},
      'P/S Ratio TTM': {'max': 8, 'min': 0.05},
      'ROE TTM': {'min': 0.10},
      'Current Ratio MRQ': {'min': 1.2},
      'Debt to Equity MRQ': {'max': 80}
    }
    
    quality_stable_criteria = {
      # Core financial
      'ROE TTM': {'min': 0.09},                       
      'Current Ratio MRQ': {'min': 1.3},              
      'Debt to Equity MRQ': {'max': 70},              

      # More flexible valuation
      # 'P/E Ratio TTM': {'max': 35, 'min': 4},  
      'P/E Ratio TTM': {'max': 40, 'min': 4}, # allowing higher for growth        
      # 'P/B Ratio': {'max': 5, 'min': 0.3},
      # 'P/S Ratio TTM': {'max': 3.0, 'min': 0.1},            

      # Very relaxed momentum (just basic filters)
      # 'Momentum 6M (%)': {'min': 5, 'max': 50}, # ADDED - 6mo performance minimum
      'Momentum 3M (%)': {'min': -10, 'max': 40},
      'Momentum 1M (%)': {'min': -8, 'max': 30},
      
      # Trend confirmation                                    
      'Price vs 50MA (%)': {'min': -15},
      'Price vs 200MA (%)': {'min': -20},       
      
      # Relative Strength
      'Relative Strength (%)': {'min': 30},       

      # Profitability and efficiency
      'Operating Margin MRQ': {'min': 0.08},
      'Profit Margin': {'min': 0.05}, 
      'Revenue Growth YOY': {'min': 0.05}, 

      # Basic volume only
      'Avg Volume 3M': {'min': 200000}

      # Current price / Target price ratio < 1 (meaning current price < target price)
      # 'Price to Target Ratio': {'max': 0.9}
    }
    
    # VALUE-QUALITY TIER - NEW ADDITION
    # Deep value + quality overlay to avoid value traps
    # Based on research: FCF yield >8%, Piotroski F-Score, ROE >15%
    value_quality_criteria = {
      # Deep value metrics
      'P/E Ratio TTM': {'max': 15, 'min': 5},            # Conservative valuation
      'P/B Ratio': {'max': 2.0, 'min': 0.3},             # Book value discount
      'P/S Ratio TTM': {'max': 2.0, 'min': 0.1},         # Sales discount
      'Price to Target Ratio': {'max': 0.85},            # 15%+ upside to analysts
      
      # Quality requirements (avoid value traps)
      'ROE TTM': {'min': 0.12},                          # Decent returns
      'Operating Margin MRQ': {'min': 0.10},             # Operational efficiency
      'Profit Margin': {'min': 0.06},                    # Actual profitability
      'Current Ratio MRQ': {'min': 1.5},                 # Liquidity
      'Debt to Equity MRQ': {'max': 70},                 # Manageable debt
      
      # Stability indicators
      'Beta': {'max': 1.3},                              # Lower volatility
    }
    
    # GARP TIER - NEW ADDITION (Growth At Reasonable Price - Peter Lynch)
    # PEG <1.0 is the key metric
    garp_criteria = {
      # Growth at reasonable price (if PEG available)
      # Note: PEG calculation may need to be added to data collection
      # 'PEG Ratio': {'max': 1.0, 'min': 0.3},          # Commented - need to add metric
      
      # Growth requirements
      'Revenue Growth YOY': {'min': 0.10},               # Double-digit growth
      'P/E Ratio TTM': {'max': 25, 'min': 8},            # Reasonable valuation
      
      # Quality overlay
      'ROE TTM': {'min': 0.15},                          # Strong returns
      'Operating Margin MRQ': {'min': 0.12},             # Efficiency
      'Profit Margin': {'min': 0.08},                    # Profitability
      'Debt to Equity MRQ': {'max': 60},                 # Manageable debt
      'Current Ratio MRQ': {'min': 1.5},                 # Liquidity
      
      # Institutional validation
      'Institutional Ownership (%)': {'min': 30, 'max': 85},  # Smart money interest
    }
    
    # Apply filters
    excellent_stocks = filter_stocks_by_criteria(df, excellent_criteria)
    good_stocks = filter_stocks_by_criteria(df, good_criteria)  
    acceptable_stocks = filter_stocks_by_criteria(df, acceptable_criteria)
    quality_stable_stocks = filter_stocks_by_criteria(df, quality_stable_criteria)
    value_quality_stocks = filter_stocks_by_criteria(df, value_quality_criteria)
    vgarp_stocks = filter_stocks_by_criteria(df, garp_criteria)
    
    return {
        'excellent': excellent_stocks,
        'good': good_stocks,
        'acceptable': acceptable_stocks,
        'quality': quality_stable_stocks,
        'value_quality': value_quality_stocks,
        'garp': vgarp_stocks,
        'criteria_explanation': {
            'excellent': 'Top-tier companies with exceptional financial metrics',
            'good': 'Solid companies with strong fundamentals', 
            'acceptable': 'Companies with adequate financial health',
            'quality': 'Stable companies with good financial health and momentum',
            'value_quality': 'Undervalued stocks with strong quality metrics to avoid value traps',
            'garp': 'Growth at a reasonable price with quality and institutional interest'
        }
    }

In [ ]:
# Run comprehensive analysis
print("🔍 COMPREHENSIVE STOCK ANALYSIS")
print("="*50)

results = comprehensive_stock_analysis(df)

# Display results by tier
for tier in ['excellent', 'good', 'quality', 'value_quality', 'garp']: # took out 'accpetable' for now
    tier_results = results[tier]
    print(f"\n🏆 {tier.upper()} STOCKS:")
    print(f"   {results['criteria_explanation'][tier]}")
    print(f"   Found: {len(tier_results['filtered_tickers'])} stocks")
    print(f"   Success Rate: {tier_results['results_summary']['success_rate']:.1f}%")
    
    if tier_results['filtered_tickers']:
        print("   Companies:")
        for ticker in tier_results['filtered_tickers']:
            company = df.loc['Company Name', ticker]
            sector = df.loc['Sector', ticker] if 'Sector' in df.index else 'N/A'
            current_price = df.loc['Current Price', ticker]
            pe_ratio = df.loc['P/E Ratio TTM', ticker]
            roe = df.loc['ROE TTM', ticker]
            print(f"     • {ticker}: {company}")
            print(f"       Sector: {sector} | Price: ${current_price} | P/E: {pe_ratio} | ROE: {roe}")
    else:
        print("   No stocks meet these criteria")
    print()

In [ ]:
df.to_csv('stock_review_100925.csv')

In [ ]:
monitor_ticker = 'FCX'

In [ ]:
pd.set_option("display.max_rows", None)
df.loc[:, monitor_ticker].to_frame()

In [ ]:
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import math

def show_price_charts(ticker, periods=[7, 21, 90, 365]):
    """
    Display price charts for specified periods for a given ticker
    
    Parameters:
    ticker (str): Stock ticker symbol (e.g., 'AAPL', 'MSFT')
    periods (list): List of days to show charts for (default: [7, 21, 90, 365])
    """
    try:
        # Calculate grid layout (2 charts per row)
        n_charts = len(periods)
        n_rows = math.ceil(n_charts / 2)
        n_cols = min(2, n_charts)
        
        # Create figure with subplots
        fig, axes = plt.subplots(n_rows, n_cols, figsize=(12, 5 * n_rows))
        
        # Handle single chart case
        if n_charts == 1:
            axes = [axes]
        elif n_rows == 1:
            axes = [axes] if n_charts == 1 else axes
        else:
            axes = axes.flatten()
        
        # Get stock data
        stock = yf.Ticker(ticker)
        
        for i, days in enumerate(periods):
            # Calculate start date
            end_date = datetime.now()
            start_date = end_date - timedelta(days=days + 10)  # Add buffer for weekends/holidays
            
            # Fetch data
            hist_data = stock.history(start=start_date, end=end_date)
            
            if hist_data.empty:
                axes[i].text(0.5, 0.5, f'No data available\nfor {ticker}', 
                           ha='center', va='center', transform=axes[i].transAxes)
                axes[i].set_title(f'{ticker} - Last {days} Days')
                continue
            
            # Get the last N days of data
            recent_data = hist_data.tail(days)
            
            # Plot the closing price
            axes[i].plot(recent_data.index, recent_data['Close'], 
                        linewidth=2, color='#1f77b4')
            axes[i].fill_between(recent_data.index, recent_data['Close'], 
                               alpha=0.3, color='#1f77b4')
            
            # Formatting
            axes[i].set_title(f'{ticker} - Last {days} Days')
            axes[i].set_xlabel('Date')
            axes[i].set_ylabel('Price ($)')
            axes[i].grid(True, alpha=0.3)
            axes[i].tick_params(axis='x', rotation=45)
            
            # Auto-scale y-axis to show fluctuations better
            if not recent_data.empty:
                price_min = recent_data['Close'].min()
                price_max = recent_data['Close'].max()
                price_range = price_max - price_min
                margin = price_range * 0.1  # Add 10% margin
                axes[i].set_ylim(price_min - margin, price_max + margin)
            
            # Add price info
            if not recent_data.empty:
                current_price = recent_data['Close'].iloc[-1]
                start_price = recent_data['Close'].iloc[0]
                change = current_price - start_price
                change_pct = (change / start_price) * 100
                
                color = 'green' if change >= 0 else 'red'
                axes[i].text(0.02, 0.98, 
                           f'Current: ${current_price:.2f}\n'
                           f'Change: ${change:.2f} ({change_pct:+.1f}%)',
                           transform=axes[i].transAxes, 
                           verticalalignment='top',
                           bbox=dict(boxstyle='round', facecolor='white', alpha=0.8),
                           color=color, fontweight='bold')
        
        # Hide empty subplots if odd number of charts
        if n_charts % 2 == 1 and n_charts > 1:
            axes[-1].set_visible(False)
        
        plt.tight_layout()
        plt.show()
        
        # Print summary info
        print(f"\n📊 {ticker} Price Summary:")
        for days in periods:
            hist_data = stock.history(start=datetime.now() - timedelta(days=days + 10))
            if not hist_data.empty:
                recent_data = hist_data.tail(days)
                if not recent_data.empty:
                    current_price = recent_data['Close'].iloc[-1]
                    start_price = recent_data['Close'].iloc[0]
                    change = current_price - start_price
                    change_pct = (change / start_price) * 100
                    print(f"  • {days}D: ${current_price:.2f} ({change_pct:+.1f}%)")
        
    except Exception as e:
        print(f"❌ Error fetching data for {ticker}: {str(e)}")


In [ ]:
show_price_charts(monitor_ticker)
# show_price_charts(monitor_ticker, periods=[7, 21, 90, 365])  # Custom periods

In [ ]:
# # Fixed Financial Health & Growth Analysis with proper numeric casting
# import pandas as pd
# import numpy as np

# def analyze_financial_health_fixed(df):
#     """
#     Comprehensive financial health and growth potential analysis with proper numeric casting
#     Returns scored DataFrame with detailed metrics
#     """
#     results = []
    
#     for ticker in df.columns:
#         stock_data = {
#             'Ticker': ticker,
#             'Company': df.loc['Company Name', ticker],
#             'Sector': df.loc['Sector', ticker] if 'Sector' in df.index else 'N/A',
#             'Current_Price': df.loc['Current Price', ticker]
#         }
        
#         # Financial Health Score (0-100)
#         health_score = 0
#         health_components = 0
        
#         # ROE (Return on Equity) - Target: >15%
#         roe = pd.to_numeric(df.loc['ROE TTM', ticker], errors='coerce')
#         if pd.notna(roe):
#             if roe > 0.20: health_score += 20
#             elif roe > 0.15: health_score += 15
#             elif roe > 0.10: health_score += 10
#             elif roe > 0.05: health_score += 5
#             stock_data['ROE'] = f"{roe:.1%}"
#             health_components += 1
#         else:
#             stock_data['ROE'] = 'N/A'
        
#         # Operating Margin - Target: >15%
#         op_margin = pd.to_numeric(df.loc['Operating Margin MRQ', ticker], errors='coerce')
#         if pd.notna(op_margin):
#             if op_margin > 0.20: health_score += 15
#             elif op_margin > 0.15: health_score += 12
#             elif op_margin > 0.10: health_score += 8
#             elif op_margin > 0.05: health_score += 4
#             stock_data['Op_Margin'] = f"{op_margin:.1%}"
#             health_components += 1
#         else:
#             stock_data['Op_Margin'] = 'N/A'
        
#         # Profit Margin - Target: >10%
#         profit_margin = pd.to_numeric(df.loc['Profit Margin', ticker], errors='coerce')
#         if pd.notna(profit_margin):
#             if profit_margin > 0.15: health_score += 15
#             elif profit_margin > 0.10: health_score += 12
#             elif profit_margin > 0.05: health_score += 8
#             elif profit_margin > 0.02: health_score += 4
#             stock_data['Profit_Margin'] = f"{profit_margin:.1%}"
#             health_components += 1
#         else:
#             stock_data['Profit_Margin'] = 'N/A'
        
#         # Current Ratio - Target: >1.5
#         current_ratio = pd.to_numeric(df.loc['Current Ratio MRQ', ticker], errors='coerce')
#         if pd.notna(current_ratio):
#             if current_ratio > 2.0: health_score += 10
#             elif current_ratio > 1.5: health_score += 8
#             elif current_ratio > 1.2: health_score += 5
#             elif current_ratio > 1.0: health_score += 2
#             stock_data['Current_Ratio'] = f"{current_ratio:.2f}"
#             health_components += 1
#         else:
#             stock_data['Current_Ratio'] = 'N/A'
        
#         # Debt to Equity - Target: <50%
#         debt_equity = pd.to_numeric(df.loc['Debt to Equity MRQ', ticker], errors='coerce')
#         if pd.notna(debt_equity):
#             if debt_equity < 20: health_score += 15
#             elif debt_equity < 40: health_score += 12
#             elif debt_equity < 60: health_score += 8
#             elif debt_equity < 80: health_score += 4
#             stock_data['Debt_Equity'] = f"{debt_equity:.1f}%"
#             health_components += 1
#         else:
#             stock_data['Debt_Equity'] = 'N/A'
        
#         # Revenue Growth - Target: >5%
#         rev_growth = pd.to_numeric(df.loc['Revenue Growth YOY', ticker], errors='coerce')
#         if pd.notna(rev_growth):
#             if rev_growth > 0.20: health_score += 15
#             elif rev_growth > 0.10: health_score += 12
#             elif rev_growth > 0.05: health_score += 8
#             elif rev_growth > 0.00: health_score += 4
#             stock_data['Revenue_Growth'] = f"{rev_growth:.1%}"
#             health_components += 1
#         else:
#             stock_data['Revenue_Growth'] = 'N/A'
        
#         # P/E Ratio - reasonable valuation
#         pe_ratio = pd.to_numeric(df.loc['P/E Ratio TTM', ticker], errors='coerce')
#         if pd.notna(pe_ratio) and pe_ratio > 0:
#             if 8 <= pe_ratio <= 20: health_score += 10
#             elif 6 <= pe_ratio <= 25: health_score += 8
#             elif 4 <= pe_ratio <= 30: health_score += 5
#             stock_data['PE_Ratio'] = f"{pe_ratio:.1f}"
#             health_components += 1
#         else:
#             stock_data['PE_Ratio'] = 'N/A'
        
#         # Normalize health score
#         if health_components > 0:
#             stock_data['Health_Score'] = round((health_score / health_components), 1)
#         else:
#             stock_data['Health_Score'] = 0
        
#         # Growth potential score
#         growth_score = 0
#         growth_components = 0
        
#         # Revenue growth (weighted heavily)
#         if pd.notna(rev_growth):
#             if rev_growth > 0.15: growth_score += 25
#             elif rev_growth > 0.10: growth_score += 20
#             elif rev_growth > 0.05: growth_score += 15
#             elif rev_growth > 0.00: growth_score += 10
#             growth_components += 1
        
#         # Price momentum (1Y)
#         momentum_1y = pd.to_numeric(df.loc['Momentum 1Y (%)', ticker], errors='coerce')
#         if pd.notna(momentum_1y):
#             if momentum_1y > 30: growth_score += 20
#             elif momentum_1y > 15: growth_score += 15
#             elif momentum_1y > 5: growth_score += 10
#             elif momentum_1y > -10: growth_score += 5
#             stock_data['Momentum_1Y'] = f"{momentum_1y:.1f}%"
#             growth_components += 1
#         else:
#             stock_data['Momentum_1Y'] = 'N/A'
        
#         # Price vs 200MA (trend strength)
#         price_200ma = pd.to_numeric(df.loc['Price vs 200MA (%)', ticker], errors='coerce')
#         if pd.notna(price_200ma):
#             if price_200ma > 20: growth_score += 15
#             elif price_200ma > 10: growth_score += 12
#             elif price_200ma > 0: growth_score += 8
#             elif price_200ma > -15: growth_score += 4
#             stock_data['Price_vs_200MA'] = f"{price_200ma:.1f}%"
#             growth_components += 1
#         else:
#             stock_data['Price_vs_200MA'] = 'N/A'
        
#         # Price to Target ratio (undervalued vs analyst targets)
#         price_target = pd.to_numeric(df.loc['Price to Target Ratio', ticker], errors='coerce')
#         if pd.notna(price_target):
#             if price_target < 0.8: growth_score += 15  # Significantly undervalued
#             elif price_target < 0.9: growth_score += 12
#             elif price_target < 1.0: growth_score += 8
#             elif price_target < 1.1: growth_score += 4
#             stock_data['Price_Target_Ratio'] = f"{price_target:.2f}"
#             growth_components += 1
#         else:
#             stock_data['Price_Target_Ratio'] = 'N/A'
        
#         # Normalize growth score
#         if growth_components > 0:
#             stock_data['Growth_Score'] = round((growth_score / growth_components), 1)
#         else:
#             stock_data['Growth_Score'] = 0
        
#         # Combined score
#         stock_data['Combined_Score'] = round((stock_data['Health_Score'] + stock_data['Growth_Score']) / 2, 1)
        
#         results.append(stock_data)
    
#     return pd.DataFrame(results)

# # Run the FIXED analysis
# print("🔧 RUNNING FIXED ANALYSIS WITH NUMERIC CASTING")
# print("=" * 60)

# analysis_df_fixed_fixed = analyze_financial_health_fixed(df)
# analysis_df_fixed_fixed = analysis_df_fixed_fixed.sort_values('Combined_Score', ascending=False)

# print("🏆 TOP STOCKS BY COMBINED FINANCIAL HEALTH & GROWTH POTENTIAL (FIXED)")
# print("=" * 80)
# print(analysis_df_fixed_fixed.head(15).to_string(index=False))

In [ ]:
# # Detailed analysis of top performers
# print("\n📈 DETAILED ANALYSIS OF TOP 10 STOCKS")
# print("=" * 80)

# top_10 = analysis_df_fixed.head(10)
# for idx, row in top_10.iterrows():
#     ticker = row['Ticker']
#     print(f"\n🔍 {ticker}: {row['Company']}")
#     print(f"   Sector: {row['Sector']} | Price: ${row['Current_Price']}")
#     print(f"   Health Score: {row['Health_Score']}/14.3 | Growth Score: {row['Growth_Score']}/20.0")
#     print(f"   Combined Score: {row['Combined_Score']}/17.2")
    
#     # Key financial metrics
#     metrics = []
#     if row['ROE'] != 'N/A': metrics.append(f"ROE: {row['ROE']}")
#     if row['Op_Margin'] != 'N/A': metrics.append(f"Op Margin: {row['Op_Margin']}")
#     if row['Profit_Margin'] != 'N/A': metrics.append(f"Profit Margin: {row['Profit_Margin']}")
#     if row['Revenue_Growth'] != 'N/A': metrics.append(f"Revenue Growth: {row['Revenue_Growth']}")
#     if row['Momentum_1Y'] != 'N/A': metrics.append(f"1Y Momentum: {row['Momentum_1Y']}")
#     if row['Price_Target_Ratio'] != 'N/A': metrics.append(f"Price/Target: {row['Price_Target_Ratio']}")
    
#     print(f"   Key Metrics: {' | '.join(metrics)}")

# # Additional insights on best growth vs value opportunities  
# print(f"\n🚀 BEST GROWTH OPPORTUNITIES (Growth Score > 12)")
# growth_leaders = analysis_df_fixed[analysis_df_fixed['Growth_Score'] > 12].head(5)
# for idx, row in growth_leaders.iterrows():
#     print(f"   • {row['Ticker']}: {row['Company']} (Growth: {row['Growth_Score']}, Health: {row['Health_Score']})")

# print(f"\n💪 BEST FINANCIAL HEALTH (Health Score > 10)")
# health_leaders = analysis_df_fixed[analysis_df_fixed['Health_Score'] > 10].head(5)
# for idx, row in health_leaders.iterrows():
#     print(f"   • {row['Ticker']}: {row['Company']} (Health: {row['Health_Score']}, Growth: {row['Growth_Score']})")

# print(f"\n💎 UNDERVALUED GEMS (Price/Target < 0.95)")
# undervalued = analysis_df_fixed[analysis_df_fixed['Price_Target_Ratio'].str.replace('N/A', '1.0').astype(float) < 0.95]
# if len(undervalued) > 0:
#     for idx, row in undervalued.head(3).iterrows():
#         print(f"   • {row['Ticker']}: {row['Company']} (P/T: {row['Price_Target_Ratio']}, Combined: {row['Combined_Score']})")
# else:
#     print("   No significantly undervalued stocks found in this dataset")